# Delta Lake Optimization

**Training Objective:** Optimize Delta Lake performance and understand Change Data Feed.

**Topics Covered:**
- Small Files Problem & OPTIMIZE
- Partitioning & Z-ORDER
- Liquid Clustering
- Change Data Feed (CDF)

## Theoretical Introduction

As data grows, query performance can degrade due to several factors:
- **Small Files Problem**: Too many small files increase metadata overhead
- **Data Layout**: Data not organized for common query patterns
- **Predicate Pushdown Inefficiency**: Scanning more data than necessary

Delta Lake provides several optimization techniques:

| Technique | Description | When to Use |
|-----------|-------------|-------------|
| **OPTIMIZE** | Compacts small files into larger ones | After many small writes |
| **Partitioning** | Physical data separation by column values | High-cardinality filter columns |
| **Z-ORDER** | Co-locates related data for better pruning | Frequently filtered columns |
| **Liquid Clustering** | Modern alternative to partitioning + Z-ORDER | New tables (recommended) |

## User Isolation

In [0]:
%run ../00_setup

## Environment Configuration

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from datetime import datetime, timedelta

# Display user context
display(
    spark.createDataFrame([
        (CATALOG, BRONZE_SCHEMA, SILVER_SCHEMA, GOLD_SCHEMA)
    ], ['catalog', 'bronze_schema', 'silver_schema', 'gold_schema'])
)

# Set catalog and schema as default
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")

## Example: The Small Files Problem

**Objective:** Demonstrate how many small files impact performance and how OPTIMIZE solves it

The "small files problem" occurs when:
- Streaming jobs write many small files
- Frequent small batch inserts
- High-concurrency writes

This leads to:
- Increased metadata overhead
- Slower listing of files
- Inefficient I/O (opening/closing many files)

In [0]:
# Create a table with many small files (simulating streaming ingestion)
spark.sql(f"""
CREATE OR REPLACE TABLE {CATALOG}.{BRONZE_SCHEMA}.small_files_demo (
    id INT,
    data STRING,
    created_at TIMESTAMP
) USING DELTA
""")

# Insert data in many small batches (simulating streaming)
from pyspark.sql.functions import lit, current_timestamp
import random
import string

print("Inserting 500 small batches to simulate streaming ingestion...")

In [0]:
from pyspark.sql.functions import lit, expr, current_timestamp
import random

# 1. Configuration
total_files = 5000
rows_per_file = 2  # Average 2 records per file
total_rows = total_files * rows_per_file

# 2. Generate data in memory (no Python loop!)
df = (
    spark.range(0, total_rows)
    .withColumn("id", lit(random.randint(1, 100)))
    .withColumn("data", expr("uuid()"))
    .withColumn("created_at", current_timestamp())
)

# 3. Write with forced number of files
df.repartition(total_files).write \
    .format("delta") \
    .mode("append") \
    .saveAsTable(f"{CATALOG}.{BRONZE_SCHEMA}.small_files_demo")

print(f"Done! Created {total_files} small files in a single transaction.")

In [0]:
display(spark.table(f"{CATALOG}.{BRONZE_SCHEMA}.small_files_demo"))

In [0]:
# Check the number of files BEFORE optimization
print("=== BEFORE OPTIMIZE ===")
before_optimize = spark.sql(f"DESCRIBE DETAIL {CATALOG}.{BRONZE_SCHEMA}.small_files_demo")
display(before_optimize.select("numFiles", "sizeInBytes"))

In [0]:
# Run OPTIMIZE to compact small files
optimize_result = spark.sql(f"""
    OPTIMIZE {CATALOG}.{BRONZE_SCHEMA}.small_files_demo
""")

display(optimize_result)

In [0]:
# Check the number of files AFTER optimization
print("=== AFTER OPTIMIZE ===")
after_optimize = spark.sql(f"DESCRIBE DETAIL {CATALOG}.{BRONZE_SCHEMA}.small_files_demo")
display(after_optimize.select("numFiles", "sizeInBytes"))

## Example: Partitioning

**Objective:** Demonstrate how partitioning improves query performance for filtered queries

Partitioning physically separates data into directories based on column values. This enables:
- **Partition Pruning**: Skip entire partitions that don't match the filter
- **Parallel Processing**: Process partitions independently
- **Efficient Deletes/Updates**: Only touch affected partitions

**Best Practices:**
- Use low-cardinality columns (e.g., date, country, status)
- Avoid over-partitioning (too many small partitions)
- Consider partition size: aim for 1GB+ per partition

In [0]:
# Create a partitioned table
spark.sql(f"""
CREATE OR REPLACE TABLE {CATALOG}.{BRONZE_SCHEMA}.orders_partitioned (
    order_id STRING,
    customer_id STRING,
    product_id STRING,
    order_date DATE,
    amount DOUBLE,
    status STRING
) 
USING DELTA
PARTITIONED BY (order_date)
""")

In [0]:
# Insert sample data across multiple dates
from datetime import date, timedelta

orders_data = []
base_date = date(2024, 1, 1)

for day_offset in range(30):  # 30 days of data
    order_date = base_date + timedelta(days=day_offset)
    for i in range(100):  # 100 orders per day
        orders_data.append((
            f"ORD-{day_offset:02d}-{i:04d}",
            f"CUST{i % 50:04d}",
            f"PROD{i % 20:03d}",
            order_date,
            round(50 + (i * 2.5), 2),
            "completed" if i % 3 != 0 else "pending"
        ))

orders_df = spark.createDataFrame(orders_data, 
    ["order_id", "customer_id", "product_id", "order_date", "amount", "status"])

orders_df.write.format("delta").mode("append").partitionBy("order_date") \
    .saveAsTable(f"{CATALOG}.{BRONZE_SCHEMA}.orders_partitioned")

print(f"Inserted {len(orders_data)} orders across 30 days")

In [0]:
# Check partitioning structure
display(spark.sql(f"DESCRIBE DETAIL {CATALOG}.{BRONZE_SCHEMA}.orders_partitioned"))

In [0]:
# Query with partition filter - only scans relevant partitions
# Check the Spark UI to see partition pruning in action
result = spark.sql(f"""
    SELECT * FROM {CATALOG}.{BRONZE_SCHEMA}.orders_partitioned
    WHERE order_date = '2024-01-15'
""")

print("Query for single date (should scan only 1 partition):")
display(result)

## Example: Z-ORDER (Data Skipping)

**Objective:** Demonstrate how Z-ORDER improves query performance by co-locating related data

Z-ORDER is a multi-dimensional clustering technique that:
- Co-locates related data in the same files
- Enables efficient data skipping based on file-level statistics
- Works best with high-cardinality columns used in filters

**When to use Z-ORDER:**
- Columns frequently used in WHERE clauses
- Columns with high cardinality
- Can specify up to 4 columns (effectiveness decreases with more)

In [0]:
spark.sql(f"DROP TABLE IF EXISTS {CATALOG}.{BRONZE_SCHEMA}.sales_zorder_demo")

In [0]:
# Create a table for Z-ORDER demonstration with auto-optimization disabled
spark.sql(f"""
CREATE OR REPLACE TABLE {CATALOG}.{BRONZE_SCHEMA}.sales_zorder_demo (
    sale_id STRING,
    customer_id STRING,
    product_id STRING,
    store_id STRING,
    sale_date DATE,
    amount DOUBLE,
    quantity INT
) USING DELTA
TBLPROPERTIES (
    delta.autoOptimize.optimizeWrite = false,
    delta.autoOptimize.autoCompact = false
)
""")

In [0]:
# Insert sample data
from datetime import date
import random

sales_data = []
for i in range(100000):  # 100K records
    sales_data.append((
        f"SALE-{i:08d}",
        f"CUST{random.randint(1, 1000):04d}",
        f"PROD{random.randint(1, 500):03d}",
        f"STORE{random.randint(1, 50):02d}",
        date(2024, random.randint(1, 12), random.randint(1, 28)),
        round(random.uniform(10, 500), 2),
        random.randint(1, 10)
    ))

sales_df = spark.createDataFrame(
    sales_data, 
    ["sale_id", "customer_id", "product_id", "store_id", "sale_date", "amount", "quantity"]
)

sales_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(
    f"{CATALOG}.{BRONZE_SCHEMA}.sales_zorder_demo"
)

In [0]:
display(sales_df)

In [0]:
# Check file statistics BEFORE Z-ORDER
print("=== BEFORE Z-ORDER ===")
display(spark.sql(f"DESCRIBE DETAIL {CATALOG}.{BRONZE_SCHEMA}.sales_zorder_demo"))

In [0]:
# Queries filtering before Z-Order
result = spark.sql(f"""
    SELECT * FROM {CATALOG}.{BRONZE_SCHEMA}.sales_zorder_demo
    WHERE customer_id = 'CUST0274' AND product_id = 'PROD367'
""")

display(result)

In [0]:
# Apply Z-ORDER on frequently filtered columns
# In this case: customer_id and product_id are common filter columns
zorder_result = spark.sql(f"""
    OPTIMIZE {CATALOG}.{BRONZE_SCHEMA}.sales_zorder_demo
    ZORDER BY (customer_id, product_id)
""")

display(zorder_result)

In [0]:
# Example query that benefits from Z-ORDER
result = spark.sql(f"""
    SELECT * FROM {CATALOG}.{BRONZE_SCHEMA}.sales_zorder_demo
    WHERE customer_id = 'CUST0274' AND product_id = 'PROD367'
""")

print("Query with Z-ORDER optimized columns (check Spark UI for data skipping):")
display(result)

## Example: Liquid Clustering (Modern Approach)

**Objective:** Introduce Liquid Clustering as a modern replacement for partitioning and Z-ORDER

**Liquid Clustering** is Databricks' newest optimization technique that:
- Automatically manages data layout
- Adapts to changing query patterns
- Eliminates need for manual partitioning decisions
- Works incrementally (no need to re-cluster entire table)

**Key Benefits:**
- No upfront partitioning decisions required
- Can change clustering columns without rewriting data
- Better performance for evolving workloads
- Simpler to manage than partitioning + Z-ORDER combo

In [0]:
spark.sql(f"DROP TABLE IF EXISTS {CATALOG}.{BRONZE_SCHEMA}.sales_liquid_clustering")

In [0]:
# Create a table with Liquid Clustering and auto-optimization disabled
spark.sql(f"""
CREATE OR REPLACE TABLE {CATALOG}.{BRONZE_SCHEMA}.sales_liquid_clustering (
    sale_id STRING,
    customer_id STRING,
    product_id STRING,
    region STRING,
    sale_date DATE,
    amount DOUBLE,
    quantity INT
) 
USING DELTA
TBLPROPERTIES (
    delta.autoOptimize.optimizeWrite = false,
    delta.autoOptimize.autoCompact = false
)

""")

In [0]:
from pyspark.sql.functions import col, rand, lit, concat, lpad, element_at, array, date_add, to_date, round

# 1. Configuration
target_files = 5000       # We want 5000 files
rows_per_file = 10        # 10 records per file
total_rows = target_files * rows_per_file # Total 50,000 records

# Array of regions for random selection
regions_list = array([lit(x) for x in ['North', 'South', 'East', 'West', 'Central']])

# 2. Data generation (no for loop!)
df = spark.range(0, total_rows).withColumnRenamed("id", "idx") \
    .withColumn("sale_id", concat(lit("SALE-"), lpad(col("idx"), 8, "0"))) \
    .withColumn("customer_id", concat(lit("CUST"), lpad((rand() * 500 + 1).cast("int"), 4, "0"))) \
    .withColumn("product_id", concat(lit("PROD"), lpad((rand() * 200 + 1).cast("int"), 3, "0"))) \
    .withColumn("region", element_at(regions_list, (rand() * 5 + 1).cast("int"))) \
    .withColumn("sale_date", date_add(to_date(lit("2024-01-01")), (rand() * 364).cast("int"))) \
    .withColumn("amount", round(rand() * 490 + 10, 2)) \
    .withColumn("quantity", (rand() * 10 + 1).cast("int")) \
    .drop("idx") # Remove helper column

# 3. Write - repartition is key
# Create table with Liquid Clustering enabled (if not exists)
# or append to existing one.
df.repartition(target_files).write \
    .format("delta") \
    .mode("append") \
    .saveAsTable(f"{CATALOG}.{BRONZE_SCHEMA}.sales_liquid_clustering")

print(f"Done! Inserted {total_rows} records in {target_files} small files.")

In [0]:
df = spark.sql(f"DESCRIBE DETAIL {CATALOG}.{BRONZE_SCHEMA}.sales_liquid_clustering")
display(df.select("numFiles"))

In [0]:
# Enable liquid clustering on an existing table by specifying clustering columns
spark.sql(f"""
ALTER TABLE {CATALOG}.{BRONZE_SCHEMA}.sales_liquid_clustering
CLUSTER BY (customer_id,product_id)
""")

In [0]:
# OPTIMIZE automatically applies Liquid Clustering
# No need to specify ZORDER - it's built into the table definition!
optimize_result = spark.sql(f"""
    OPTIMIZE {CATALOG}.{BRONZE_SCHEMA}.sales_liquid_clustering
""")

display(optimize_result)

In [0]:
# Check clustering information
display(spark.sql(f"DESCRIBE DETAIL {CATALOG}.{BRONZE_SCHEMA}.sales_liquid_clustering"))

In [0]:
# Queries filtering by clustering columns are automatically optimized
result = spark.sql(f"""
    SELECT region, COUNT(*) as sales_count, SUM(amount) as total_amount
    FROM {CATALOG}.{BRONZE_SCHEMA}.sales_liquid_clustering
    WHERE customer_id LIKE 'CUST00%' AND region = 'North'
    GROUP BY region
""")

display(result)

**Comparison: Partitioning vs Z-ORDER vs Liquid Clustering**

| Feature | Partitioning | Z-ORDER | Liquid Clustering |
|---------|-------------|---------|-------------------|
| When to choose | Low-cardinality columns | High-cardinality filter columns | General purpose (recommended) |
| Data layout | Directory per partition | Co-located in files | Automatic clustering |
| Schema change | Requires rewrite | Easy to change | Easy to change |
| Maintenance | Manual | Manual OPTIMIZE | Automatic with OPTIMIZE |
| Best for | Date/Region filters | Multi-column filters | Evolving workloads |

# Change Data Feed vs Change Data Capture

**Theoretical Introduction:**

Two terms are often confused in the data engineering world: **Change Data Feed (CDF)** and **Change Data Capture (CDC)**. Understanding the difference is crucial:

### Change Data Capture (CDC)
**What it is:** A *pattern/technique* for capturing changes from source systems (databases, APIs, etc.)

**Characteristics:**
- Source-side technology
- Captures INSERT, UPDATE, DELETE from operational databases
- Tools: Debezium, AWS DMS, Fivetran, Qlik Replicate
- Produces a stream of change events

**Example:** Capturing changes from PostgreSQL and streaming them to Kafka

### Change Data Feed (CDF)
**What it is:** A *Delta Lake feature* that records row-level changes within Delta tables

**Characteristics:**
- Delta Lake native feature
- Tracks changes that happen WITHIN Delta tables
- Provides `_change_type`, `_commit_version`, `_commit_timestamp` columns
- Enables efficient incremental processing

**Example:** Reading only the rows that changed since the last pipeline run

### How They Work Together
```
[Source DB] --CDC--> [Bronze Delta] --CDF--> [Silver Delta] --CDF--> [Gold Delta]
     ^                    ^                      ^                       ^
     |                    |                      |                       |
   CDC captures      CDF tracks            CDF tracks              CDF tracks
   source changes    Bronze changes        Silver changes          Gold changes
```

## Example: Enabling Change Data Feed

**Objective:** Enable CDF on a Delta table and understand what metadata is captured

In [0]:
# Create a table with CDF enabled from the start
spark.sql(f"""
CREATE OR REPLACE TABLE {CATALOG}.{BRONZE_SCHEMA}.cdf_demo (
    user_id STRING,
    name STRING,
    email STRING,
    status STRING,
    updated_at TIMESTAMP
) 
USING DELTA
TBLPROPERTIES (delta.enableChangeDataFeed = true)
""")

print("Table created with Change Data Feed enabled")

In [0]:
# Verify CDF is enabled
properties = spark.sql(f"SHOW TBLPROPERTIES {CATALOG}.{BRONZE_SCHEMA}.cdf_demo")
display(properties.filter(F.col("key").like("%change%")))

## Example: Generating and Tracking Changes

**Objective:** Execute various DML operations and see how CDF records them

In [0]:
# INSERT some initial data
spark.sql(f"""
INSERT INTO {CATALOG}.{BRONZE_SCHEMA}.cdf_demo VALUES
    ('U001', 'Alice', 'alice@example.com', 'active', current_timestamp()),
    ('U002', 'Bob', 'bob@example.com', 'active', current_timestamp()),
    ('U003', 'Charlie', 'charlie@example.com', 'active', current_timestamp())
""")
print("Version 1: Initial INSERT completed")

In [0]:
# UPDATE a record
spark.sql(f"""
UPDATE {CATALOG}.{BRONZE_SCHEMA}.cdf_demo
SET status = 'premium', updated_at = current_timestamp()
WHERE user_id = 'U001'
""")
print("Version 2: UPDATE completed - Alice upgraded to premium")

In [0]:
# DELETE a record
spark.sql(f"""
DELETE FROM {CATALOG}.{BRONZE_SCHEMA}.cdf_demo
WHERE user_id = 'U002'
""")
print("Version 3: DELETE completed - Bob removed")

In [0]:
# INSERT new record
spark.sql(f"""
INSERT INTO {CATALOG}.{BRONZE_SCHEMA}.cdf_demo VALUES
    ('U004', 'Diana', 'diana@example.com', 'trial', current_timestamp())
""")
print("Version 4: INSERT completed - Diana added")

## Example: Reading Change Data Feed

**Objective:** Query the Change Data Feed to see all recorded changes

In [0]:
# Read all changes from the beginning
changes = spark.read \
    .format("delta") \
    .option("readChangeFeed", "true") \
    .option("startingVersion", 0) \
    .table(f"{CATALOG}.{BRONZE_SCHEMA}.cdf_demo")

# Show changes with CDF metadata columns
display(
    changes.select(
        "user_id", "name", "status",
        "_change_type",        # insert, update_preimage, update_postimage, delete
        "_commit_version",     # Delta version number
        "_commit_timestamp"    # When the change occurred
    ).orderBy("_commit_version", "user_id")
)

**Understanding `_change_type` values:**

| Change Type | Description |
|-------------|-------------|
| `insert` | New row inserted |
| `update_preimage` | Row value BEFORE update |
| `update_postimage` | Row value AFTER update |
| `delete` | Row that was deleted |

This enables powerful incremental processing patterns - you can process only what changed!

In [0]:
# Example: Get only new inserts since version 2
new_inserts = spark.read \
    .format("delta") \
    .option("readChangeFeed", "true") \
    .option("startingVersion", 2) \
    .table(f"{CATALOG}.{BRONZE_SCHEMA}.cdf_demo") \
    .filter(F.col("_change_type") == "insert")

print("New inserts since version 2:")
display(new_inserts.select("user_id", "name", "status", "_commit_version"))

In [0]:
# Example: Get all deletions for audit purposes
deletions = spark.read \
    .format("delta") \
    .option("readChangeFeed", "true") \
    .option("startingVersion", 0) \
    .table(f"{CATALOG}.{BRONZE_SCHEMA}.cdf_demo") \
    .filter(F.col("_change_type") == "delete")

print("All deleted records (for audit):")
display(deletions.select("user_id", "name", "_commit_version", "_commit_timestamp"))

## Example: CDF for Incremental ETL

**Objective:** Demonstrate how CDF enables efficient incremental processing in ETL pipelines

Instead of reprocessing entire tables, use CDF to process only changed rows:

In [0]:
# Simulate an incremental ETL pipeline
# First run: Process all data (startingVersion = 0)
# Subsequent runs: Process only changes since last processed version

# Store the last processed version (in practice, save this to a checkpoint table)
last_processed_version = 0

# Read incremental changes
incremental_changes = spark.read \
    .format("delta") \
    .option("readChangeFeed", "true") \
    .option("startingVersion", last_processed_version) \
    .table(f"{CATALOG}.{BRONZE_SCHEMA}.cdf_demo")

# Apply transformations only to changed records
transformed = incremental_changes \
    .filter(F.col("_change_type").isin(["insert", "update_postimage"])) \
    .withColumn("processed_at", F.current_timestamp()) \
    .withColumn("email_domain", F.split(F.col("email"), "@")[1])

print("Incremental processing - only changed records:")
display(transformed.select("user_id", "name", "email_domain", "status", "_change_type", "processed_at"))

**Key Takeaways - CDF vs CDC:**

1. **CDC** captures changes FROM source systems INTO your lakehouse
2. **CDF** tracks changes WITHIN Delta Lake tables
3. Use CDC tools (Debezium, DMS) to ingest data into Bronze
4. Use CDF to build efficient incremental Silver and Gold layers
5. CDF eliminates need for expensive full-table scans in pipelines

---

# Summary

### What Has Been Achieved (Modules 4 & 5):

| Module | Key Learnings |
|---------|--------------|
| **Module 4: Operations** | Delta table creation, Schema Enforcement, Schema Evolution, Constraints, CRUD, Time Travel |
| **Module 5: Optimization** | Small Files Problem, Partitioning, Z-ORDER, Liquid Clustering, Change Data Feed (CDF) |

### Key Takeaways:

1. **Delta Lake = Data Lake + ACID**: Combines Data Lake flexibility with transactional reliability
2. **Schema Evolution safely**: Additive changes are automatic, breaking changes require planning
3. **Time Travel + Copy-on-Write**: Every version is preserved, enabling rollback and audit
4. **VACUUM trade-off**: Storage optimization vs Time Travel capability
5. **Optimization matters**: Choose the right technique (Partitioning, Z-ORDER, Liquid Clustering)
6. **CDF ≠ CDC**: CDC ingests from sources, CDF tracks Delta Lake changes

### Quick Reference - Key Commands:

| Operation | SQL | PySpark |
|-----------|-----|---------|
| Create Delta Table | `CREATE TABLE USING DELTA` | `df.write.format("delta").saveAsTable()` |
| Time Travel | `SELECT * FROM table VERSION AS OF 1` | `.option("versionAsOf", 1)` |
| Restore | `RESTORE TABLE table TO VERSION AS OF 1` | N/A |
| MERGE | `MERGE INTO target USING source` | `DeltaTable.forName().merge()` |
| Optimize | `OPTIMIZE table` | N/A |
| Z-ORDER | `OPTIMIZE table ZORDER BY (col)` | N/A |
| VACUUM | `VACUUM table RETAIN X HOURS` | N/A |
| History | `DESCRIBE HISTORY table` | N/A |
| Enable CDF | `ALTER TABLE SET TBLPROPERTIES (delta.enableChangeDataFeed = true)` | N/A |
| Read CDF | N/A | `.option("readChangeFeed", "true")` |

---

# Resource Cleanup

Clean up resources created during the notebook:

In [0]:
# Optional test resource cleanup
# NOTE: Run only if you want to delete all created data

# Tables to clean up:
cleanup_tables = [
    "customers_delta",
    "orders_modern", 
    "time_travel_demo",
    "small_files_demo",
    "orders_partitioned",
    "sales_zorder_demo",
    "sales_liquid_clustering",
    "cdf_demo"
]


In [0]:
# Uncomment below to execute cleanup:
# for table in cleanup_tables:
#     spark.sql(f"DROP TABLE IF EXISTS {CATALOG}.{BRONZE_SCHEMA}.{table}")
#     print(f"Dropped: {table}")

# spark.sql("DROP VIEW IF EXISTS customer_updates")
# spark.catalog.clearCache()

# print("All resources cleaned up!")